<a href="https://colab.research.google.com/github/petersim1/deep-image-prior/blob/master/simple_linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os

In [4]:
cd drive/MyDrive/Semester\ 3/Bayesian\ ML/Project/flikr8k_sized

/content/drive/MyDrive/Semester 3/Bayesian ML/Project/flikr8k_sized


In [5]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [6]:
images = []
for ind, f in enumerate(os.listdir('.')):
  if f != '.DS_Store':
    im = Image.open('./'+f) 
    im = np.asarray(im)
    images.append(im)
  if ind == 5:
    break

In [7]:
for i in images:
  print(i.shape, np.max(i))

(333, 500, 3) 255
(333, 500, 3) 255
(333, 500, 3) 255
(333, 500, 3) 255
(333, 500, 3) 255
(333, 500, 3) 255


In [8]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import MNIST
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os

In [9]:
from torch.utils.data import DataLoader, Dataset
class ImageDataset(Dataset):
    def __init__(self, images, mode='train'):
        split = int(len(images)*.8)
        if mode == 'train':
            self.images = images[:split]
        elif mode == 'val':
            self.images = images[split:]
            
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        im = torch.from_numpy(np.moveaxis(images[idx], -1, 0)).float()
        label = torch.Tensor([0])
        return im, label

In [10]:
train_dataset = ImageDataset(images, 'train')
valid_dataset = ImageDataset(images, 'val')

train_loader = DataLoader(dataset = train_dataset, batch_size = 1, shuffle = True)
val_loader = DataLoader(dataset = valid_dataset, batch_size = 1, shuffle = False)

In [11]:
# Reconstruction + KL divergence losses summed over all elements and batch

def loss_function(x_hat, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(
        x_hat, x.view(-1, 499500), reduction='sum'
    )
    KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))

    return BCE + KLD

In [12]:
class bigVAE(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(499500, 200), #500x333x3
            nn.ReLU(),
            nn.Linear(200, 50),
        )

        self.decoder = nn.Sequential(
            nn.Linear(25, 200),
            nn.ReLU(),
            nn.Linear(200, 499500),
            nn.Sigmoid(),
        )

    def reparameterise(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        mu_logvar = self.encoder(x.view(-1, 499500)).view(-1, 2, 25)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]
        z = self.reparameterise(mu, logvar)
        return self.decoder(z), mu, logvar

    def initialize_weights(self):
        for m in self.modules():
#             print(m)
            if isinstance(m, Sequential):
                for elem in m:
                    if isinstance(elem, Linear):
                        torch.nn.init.zeros_(elem.weight)
            elif isinstance(m, Linear):
                torch.nn.init.zeros_(elem.weight)

device = torch.device("cpu")#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = bigVAE().to(device)

In [13]:
print(device)

cpu


In [14]:
learning_rate = 1e-3

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [15]:
# Training and testing the VAE

epochs = 10
codes = dict(μ=list(), logσ2=list())
for epoch in range(0, epochs + 1):
    # Training
    if epoch > 0:  # test untrained net first
        model.train()
        train_loss = 0
        for x, _ in train_loader:
            x = x.to(device)
            # ===================forward=====================
            x_hat, mu, logvar = model(x)
            print(torch.max(x_hat), torch.max(x))
            loss = loss_function(x_hat, x, mu, logvar)
            train_loss += loss.item()
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')
    
    # Testing
    
    means, logvars = list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for x, y in val_loader:
            x = x.to(device)
            # ===================forward=====================
            x_hat, mu, logvar = model(x)
            test_loss += loss_function(x_hat, x, mu, logvar).item()
            # =====================log=======================
            means.append(mu.detach())
            logvars.append(logvar.detach())
    # ===================log========================
    # codes['μ'].append(torch.cat(means))
    # codes['logσ2'].append(torch.cat(logvars))
    test_loss /= len(val_loader.dataset)
    print(f'====> Test set loss: {test_loss:.4f}')
    # display_images(x, x_hat, 1, f'Epoch {epoch}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  


====> Test set loss: 143707757053109137978603601920.0000
tensor(1., grad_fn=<MaxBackward1>) tensor(255.)
tensor(1., grad_fn=<MaxBackward1>) tensor(255.)
tensor(1., grad_fn=<MaxBackward1>) tensor(255.)
tensor(1., grad_fn=<MaxBackward1>) tensor(255.)
====> Epoch: 1 Average loss: 19440054319225333031956300594937856.0000
====> Test set loss: inf
tensor(nan, grad_fn=<MaxBackward1>) tensor(255.)


RuntimeError: ignored